<a href="https://colab.research.google.com/github/DEEPAK4151/python-practice-colab1/blob/main/homeopathic_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

i am trying to make a ds model to predict  homeopathic medicines by symptoms disease and other things

In [1]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:

df =pd.read_csv("sample_data/homeopathy_remedy_dataset_1000_rows.csv")

In [35]:
df

,remedy,symptoms,disease,potency,source,remedy_potency
0,Pulsatilla,chronic eruptions; swelling; acne; itching; ve...,hormonal,6X,synthetic materia medica,Pulsatilla | 6X
1,Mercurius Solubilis,dull skin; acne; vesicular eruptions; swelling...,allergy,30C,synthetic materia medica,Mercurius Solubilis | 30C
2,Sulphur,acne; cracked skin; vesicular eruptions; redness,chronic,Q,synthetic materia medica,Sulphur | Q
3,Hepar Sulph,pimples; cracked skin; eczema; chronic eruptio...,hormonal,6X,synthetic materia medica,Hepar Sulph | 6X
4,Kali Bromatum,allergic rash; redness; sensitive skin; facial...,chronic,6X,synthetic materia medica,Kali Bromatum | 6X
...,...,...,...,...,...,...
995,Kali Bromatum,itching; pimples; hormonal acne; eczema; pigme...,general,6X,synthetic materia medica,Kali Bromatum | 6X
996,Silicea,dry skin; eczema; chronic eruptions; vesicular...,acute inflammation,Q,synthetic materia medica,Silicea | Q
997,Kali Bromatum,cracked skin; rash; sensitive skin; swelling; ...,acute inflammation,200C,synthetic materia medica,Kali Bromatum | 200C
998,Belladonna,hormonal acne; dull skin; eczema; cracked skin...,hormonal,200C,synthetic materia medica,Belladonna | 200C


In [7]:
df['symptoms']=df['symptoms'].str.lower()


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df['symptoms'])

In [9]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6406 stored elements and shape (1000, 23)>

In [19]:
y_disease = df['disease']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_disease, test_size=0.2, random_state=42
)



In [20]:
from sklearn.ensemble import RandomForestClassifier

disease_model = RandomForestClassifier(n_estimators=200)

disease_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

In [21]:
from sklearn.metrics import classification_report

y_pred = disease_model.predict(X_test)

print(classification_report(y_test, y_pred))

                    precision    recall  f1-score   support

acute inflammation       0.28      0.35      0.31        31
           allergy       0.20      0.22      0.21        36
           chronic       0.41      0.23      0.30        30
           general       0.09      0.08      0.09        37
          hormonal       0.10      0.11      0.10        37
              skin       0.16      0.17      0.17        29

          accuracy                           0.19       200
         macro avg       0.21      0.20      0.20       200
      weighted avg       0.20      0.19      0.19       200



In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

y_remedy = mlb.fit_transform(
    df['remedy_potency'].apply(lambda x: [x])
)

In [23]:
from sklearn.multiclass import OneVsRestClassifier

remedy_model = OneVsRestClassifier(
    RandomForestClassifier(n_estimators=200)
)

remedy_model.fit(X, y_remedy)

OneVsRestClassifier(estimator=RandomForestClassifier(n_estimators=200))

In [28]:
import numpy as np

def predict_full(symptom_text, top_k=3):
    symptom_text = symptom_text.lower()
    vec = vectorizer.transform([symptom_text])

    # disease prediction
    disease = disease_model.predict(vec)[0]

    # remedy probabilities
    probs = remedy_model.predict_proba(vec)[0]

    # get top-k indices
    top_indices = np.argsort(probs)[-top_k:][::-1]

    remedies = [mlb.classes_[i] for i in top_indices]

    return {
        "predicted_disease": disease,
        "recommended_remedies": remedies
    }

In [36]:
result = predict_full("chronic eruptions; swelling; acne; itching; vesicular eruptions; eczema")

print(result)

{'predicted_disease': 'hormonal', 'recommended_remedies': ['Pulsatilla | 6X', 'Mercurius Solubilis | 30C', 'Apis Mellifica | Q']}


In [38]:
import joblib

joblib.dump(disease_model, "disease_model.pkl")
joblib.dump(remedy_model, "remedy_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")
joblib.dump(mlb, "mlb.pkl")

print("Models saved!")

Models saved!


In [42]:
da=df['symptoms']
da


,symptoms
0,chronic eruptions; swelling; acne; itching; ve...
1,dull skin; acne; vesicular eruptions; swelling...
2,acne; cracked skin; vesicular eruptions; redness
3,pimples; cracked skin; eczema; chronic eruptio...
4,allergic rash; redness; sensitive skin; facial...
...,...
995,itching; pimples; hormonal acne; eczema; pigme...
996,dry skin; eczema; chronic eruptions; vesicular...
997,cracked skin; rash; sensitive skin; swelling; ...
998,hormonal acne; dull skin; eczema; cracked skin...
